In [1]:
import cv2 as cv 
import numpy as np

In [6]:
def camera_calibration(verbose = False):
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    # In this case, we don't know square size since we didn't take those images, so we pass in terms of square size
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    
    objpoints = [] # 3d point in real world space, 
    imgpoints = [] # 2d points in image plane.
    
    
    # define a video capture object
    vid = cv.VideoCapture('Data\calibration.mp4')
    ret, img = vid.read()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    count = 0
    while(True):

        # Capture the video frame
        # by frame
        ret, img = vid.read()

        if not ret:
            print(f'No frames grabbed!, total frame {count}')
            break

        # we count the number of frame 
        # we wanted to pick 40 frames from the video sequence
        # we pick them uniformaly 
        # total frame : 2419
        if count % 60 == 0:

            # convert to gray scale
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            
            #find the chess board corners, It returns the corner points
            #                                            what kind of pattern we are looking for,
            ret, corners = cv.findChessboardCorners(gray, (9,6), None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                objpoints.append(objp)
                # Once we find the corners, we can increase their accuracy using cv.cornerSubPix()
                corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)
                if verbose:
                     # Draw and display the corners
                    cv.drawChessboardCorners(img, (9,6), corners2, ret)
                    cv.namedWindow("win1", cv.WINDOW_NORMAL )
                    cv.imshow('win1', img)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv.waitKey(500) & 0xFF == ord('q'):
            break

        count +=1 

    # After the loop release the cap object
    vid.release()


    # Destroy all the windows
    cv.destroyAllWindows()
    # Now that we have our object points and image points, we are ready to go for calibration.
    # calibartion calibartion
    # return camera matrix, distortion coefficients, rotation and translation vectors
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    # save file 
    #np.save("Data/camera_matrix", mtx)
    #np.save("Data/dist_coef", dist)
    return mtx,dist 

In [7]:
mtx, dist = camera_calibration(verbose = True)

In [12]:
mtx

array([[1.66488939e+03, 0.00000000e+00, 9.52968875e+02],
       [0.00000000e+00, 1.66727534e+03, 5.29233098e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [8]:
dist

array([[ 1.23764716e-01, -1.59125045e-01,  2.62414401e-05,
        -1.87954245e-03,  6.36970080e-02]])